In [3]:
import logging
import copy
import itertools
from qiskit.quantum_info import Pauli
from pyscf import gto, scf, ao2mo
from pyscf.lib import param
from scipy import linalg as scila
from pyscf.lib import logger as pylogger
# from qiskit.chemistry import AquaChemistryError
from qiskit.chemistry import QMolecule
import numpy as np
from qiskit.aqua.operators import WeightedPauliOperator as WPO

from qiskit.aqua.algorithms import ExactEigensolver
# import gse_algo as ga
import scipy
from pyscf.scf.hf import get_ovlp
# from symmetries import find_symmetry_ops
from qiskit.chemistry import FermionicOperator
from int_func import qmol_func

logger = logging.getLogger(__name__)
# import int_func
# from qiskit import BasicAer, Aer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms.adaptive import VQE
from qiskit.aqua.algorithms.classical import ExactEigensolver
from qiskit.aqua.components.optimizers import SLSQP, CG
from qiskit.aqua.operators import Z2Symmetries, WeightedPauliOperator
from qiskit.chemistry.core import Hamiltonian, TransformationType, QubitMappingType
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.aqua_extensions.components.variational_forms import UCCSD
from qiskit.chemistry.aqua_extensions.components.initial_states import HartreeFock
from qiskit.chemistry import set_qiskit_chemistry_logging
from qiskit.aqua.utils import CircuitCache
from qiskit.chemistry.drivers import PSI4Driver,PySCFDriver, UnitsType

import logging
import itertools


Numpy 1.16 has memory leak bug  https://github.com/numpy/numpy/issues/13808
It is recommended to downgrade to numpy 1.15 or older


In [19]:
# driver = PySCFDriver(atom='N 0.0 0.0 0.1493220; H 0.0 0.9474830 -0.3484190; H 0.8205440 -0.4737420 -0.3484190; H -0.8205440 -0.4737420 -0.3484190',
#              unit=UnitsType.ANGSTROM,
#                          charge=0, spin=0, basis='sto3g')
num_spin_down_elec=5
num_spin_up_elec=5
num_particles=10
# driver = PySCFDriver(atom='Li 0 0 0; H 1.5949 0.0000 0.000',
#              unit=UnitsType.ANGSTROM,
#                          charge=0, spin=0, basis='sto3g')
# num_spin_down_elec=2
# num_spin_up_elec=2
# num_particles=4
driver = PySCFDriver(atom='O 0.0 0.0 0.0;H 0.757 0.586 0.0; H -0.757 0.586 0.0',
             unit=UnitsType.ANGSTROM,
                         charge=0, spin=0, basis='sto3g')
# num_spin_down_elec=3
# num_spin_up_elec=3
# num_particles=6
driver = PySCFDriver(atom='Be 0.0 0.0 0.0;H 1.291 0.0 0.0; H -1.291 0.0 0.0',
             unit=UnitsType.ANGSTROM,
                         charge=0, spin=0, basis='sto3g')

molecule = driver.run()
one_b = molecule.one_body_integrals
two_b = molecule.two_body_integrals
fer_op = FermionicOperator(h1=one_b, h2=two_b)
qub_op = fer_op.mapping('jordan_wigner')

num_qubs = np.shape(fer_op.h1)[0]
# Getting the Hartree Fock state
intial_state = HartreeFock(num_qubs, num_qubs, num_particles,
qubit_mapping='jordan_wigner')
HF_str = intial_state.bitstr
print('Hartree-Fock state')
print(HF_str)

# The following method creates the symmetries and cliffords
# corresponding to the two qubit reduction and then infers the
# tapering off coefficient.

def particle_cons_symm(qub_op, num_qubs, num_spin_up_elec, num_spin_down_elec):
    sym_str_1 = 'I'*int(num_qubs/2)+'Z'*int(num_qubs/2)
    sym_str_2 = 'Z'*int(num_qubs/2)+'I'*int(num_qubs/2)
    symmetries = [Pauli.from_label(sym_str_1),Pauli.from_label(sym_str_2)]
    sq_list = [0,int(num_qubs/2)]
    cl_1 = 'I'*int(num_qubs-1)+'X'
    cl_2 = 'I'*int(num_qubs/2-1)+'X'+'I'*int(num_qubs/2)
    sq_paulis = [Pauli.from_label(cl_1), Pauli.from_label(cl_2)]
    symm_list=[]
    cliffords = []
    cliffords.append(WPO([[1 / np.sqrt(2), symmetries[0]], [1 / np.sqrt(2), Pauli.from_label(cl_1)]]))
    cliffords.append(WPO([[1 / np.sqrt(2), symmetries[1]], [1 / np.sqrt(2), Pauli.from_label(cl_2)]]))    
    if num_spin_up_elec%2:
        coeff_1 = -1
    else:
        coeff_1 = 1
    if num_spin_down_elec%2:
        coeff_2 = -1
    else:
        coeff_2 = 1
    return [symmetries, sq_paulis, sq_list, cliffords, coeff_1, coeff_2]
    
[symmetries, sq_paulis, sq_list, cliffords, coeff_1, coeff_2] = particle_cons_symm(qub_op, num_qubs, num_spin_up_elec,num_spin_down_elec)
z2symm = Z2Symmetries(symmetries, sq_paulis, sq_list, [coeff_1,coeff_2])
# Two qubit taperring, qubit operator
the_tapered_ops = z2symm.taper(qub_op)
print('Coefficients for the two qubit tapering')
print(coeff_1)
print(coeff_2)


two_qubit_reduction only works with parity qubit mapping but you have jordan_wigner. We switch two_qubit_reduction to False.


Hartree-Fock state
[False False  True  True  True  True  True False False  True  True  True
  True  True]
Coefficients for the two qubit tapering
-1
-1


In [5]:
# Printing the symmetries for two qubit tapering along with cliffords
Print_ops = True
if Print_ops:
    for symm in symmetries:
    #     symm = Pauli.from_label(symm_str) 
        print(symm.to_label())
        sym_la = symm.to_label()
        ind = [13-i for i, a in enumerate(sym_la) if a == 'Z']
        print(ind)
    print('single qubit operators found:')
    for sq in sq_paulis:
        print(sq.to_label())
    print('cliffords found:')
    for clifford in cliffords:
        print(clifford.print_details())
        print('single-qubit list: {}'.format(sq_list))



IIIIIIIZZZZZZZ
[6, 5, 4, 3, 2, 1, 0]
ZZZZZZZIIIIIII
[13, 12, 11, 10, 9, 8, 7]
single qubit operators found:
IIIIIIIIIIIIIX
IIIIIIXIIIIIII
cliffords found:
IIIIIIIZZZZZZZ	(0.7071067811865475+0j)
IIIIIIIIIIIIIX	(0.7071067811865475+0j)

single-qubit list: [0, 7]
ZZZZZZZIIIIIII	(0.7071067811865475+0j)
IIIIIIXIIIIIII	(0.7071067811865475+0j)

single-qubit list: [0, 7]


In [6]:
# Finding all the symmetries using the find_Z2_symmetries:
z2_symmetries = Z2Symmetries.find_Z2_symmetries(qub_op)

# taper_coef = []
# for sym in z2_symmetries.symmetries:
#     print(sym.to_label())
#     ind = [i for i, a in enumerate(sym.to_label()) if a == 'Z']
#     print(ind)
#     coef=1
#     for i in ind:
#         if HF_str[i]:
#             coef=coef*(-1)
#     taper_coef.append(coef)
# print('The coefficients for various symmetries are:')
# print(taper_coef)
taper_coef = z2_symmetries.taper_coef_from_HF(HF_str)
print(taper_coef)
the_tapered_ops_1 = z2_symmetries.taper(qub_op,taper_coef)

ZIZIZIIIZIZIZZ
[0, 2, 4, 8, 10, 12, 13]
IIZIIIIIIZIIII
[2, 9]
ZIIIZIIZIIIZII
[0, 4, 7, 11]
ZZZZZZZIIIIIII
[0, 1, 2, 3, 4, 5, 6]
[-1, 1, 1, -1]


In [7]:
# running exact eigensolver without tapering off
ee = ExactEigensolver(qub_op, k=1)
ee_result = ee.run()
#   Get the first eigenvalue of the Hamiltonian spectrum
ref_min_eigvals = ee_result['eigvals'][0]
# This is the reference value from Hamiltonian
print('Eigenvalue of the full Ham')
print(ref_min_eigvals)

# running exact eigensolver after tapering off qubits (all the symmetries
# not just two qubit tapering)
ee = ExactEigensolver(the_tapered_ops_1, k=1)
ee_result = ee.run()
temp_min_eigvals = ee_result['eigvals'][0]

print('Eigenvalue from taperred off qubit operator')
print(temp_min_eigvals)


Eigenvalue of the full Ham
(-84.20635059311824+2.0276865972440847e-17j)
Eigenvalue from taperred off qubit operator
(-84.20635059311738-9.220500276730757e-16j)


In [9]:

z2_symmetries = Z2Symmetries.find_Z2_symmetries(qub_op)
the_tapered_ops = z2_symmetries.taper(qub_op)



eig value: (-83.70438788002659+1.3601842774423198e-15j)
eig value: (-83.7997920528714+3.593741817559907e-15j)
eig value: (-83.59296196712512-5.323082093473388e-16j)
eig value: (-83.62605106361443-2.0489991738115307e-15j)
eig value: (-83.88869569130338+1.2391902278836835e-15j)
eig value: (-83.80797218138608-4.6784444812360026e-15j)
eig value: (-83.70194564964162+2.6332951143897e-15j)
eig value: (-83.26024225124831+6.172714070348038e-16j)
eig value: (-83.79979205287093-2.0904584620894747e-15j)
eig value: (-84.20635059311745-1.320761771169137e-15j)
eig value: (-83.62605106361444+7.082585503423377e-17j)
eig value: (-83.59296196712579+1.3867864346658098e-16j)
eig value: (-83.80797218138684+2.9408133947124704e-16j)
eig value: (-83.88869569130297+2.264906367331442e-15j)
eig value: (-83.26024225124682-3.0118941735545177e-16j)
eig value: (-83.70194564964194-2.342467282791801e-15j)


In [16]:
# print(type(the_tapered_ops[0]))
# for op in the_tapered_ops:
i=0
for coeff in itertools.product([1, -1], repeat=len(z2_symmetries._sq_list)):
    op = the_tapered_ops[i]
    ee = ExactEigensolver(op, k=1)
    ee_result = ee.run()
    temp_min_eigvals = ee_result['eigvals'][0]
    if np.isclose(temp_min_eigvals, ref_min_eigvals, rtol=1e-8):
        correct_sector = list(coeff)
#     print("eig value: {}".format(temp_min_eigvals))
    print("at sector {}: eig value: {}; reference: {}".format(list(coeff), temp_min_eigvals, ref_min_eigvals.real))
    i=i+1

# for taper_coeff in itertools.product([1, -1], repeat=len(sq_list)):
    
#     z2symm = Z2Symmetries(symmetries, sq_paulis, sq_list, taper_coeff)
#     the_tapered_ops = z2symm.taper(qub_op)
# #     tapered_qubit_op = Operator.qubit_tapering(qub_op, cliffords, sq_list, list(taper_coeff))
#     ee = ExactEigensolver(the_tapered_ops, k=1)
#     ee_result = ee.run()
#     temp_min_eigvals = ee_result['eigvals'][0]
#     if np.isclose(temp_min_eigvals, ref_min_eigvals, rtol=1e-8):
#         correct_sector = list(taper_coeff)
#     print("at sector {}: eig value: {}; reference: {}".format(list(taper_coeff), temp_min_eigvals, ref_min_eigvals.real))

at sector [1, 1, 1, 1]: eig value: (-83.70438788002659-1.5626493571162844e-15j); reference: -84.20635059311824
at sector [1, 1, 1, -1]: eig value: (-83.79979205287185-6.62381996480299e-16j); reference: -84.20635059311824
at sector [1, 1, -1, 1]: eig value: (-83.59296196712566+6.195792850307504e-16j); reference: -84.20635059311824
at sector [1, 1, -1, -1]: eig value: (-83.62605106361441+2.5628463466869208e-17j); reference: -84.20635059311824
at sector [1, -1, 1, 1]: eig value: (-83.88869569130345+1.5734299501012115e-15j); reference: -84.20635059311824
at sector [1, -1, 1, -1]: eig value: (-83.80797218138716-1.0918030148760064e-16j); reference: -84.20635059311824
at sector [1, -1, -1, 1]: eig value: (-83.70194564964211+2.4543621923987212e-15j); reference: -84.20635059311824
at sector [1, -1, -1, -1]: eig value: (-83.26024225124742-5.975974627629248e-16j); reference: -84.20635059311824
at sector [-1, 1, 1, 1]: eig value: (-83.79979205287134-1.3796838047120394e-15j); reference: -84.2063505

In [17]:
correct_sector

[-1, 1, 1, -1]